# Crawling Web

### Crwaling Web Adalah

Jadi crawling web adalah sebuah proses dimana kita akan mengambil sebuah data dari sebuah web menggunakan code. Untuk crawling web saya menggunakan file python biasa yang harus menggunakan visual studio code, berikut ini adalah code nya:

In [57]:
import scrapy


class QuotesSpider(scrapy.Spider):
    name = "quotes"

    def start_requests(self):
        x = 100000
        for i in range (1,500):
            x +=1
            urls = [
                'https://pta.trunojoyo.ac.id/welcome/detail/070411'+str(x),
                'https://pta.trunojoyo.ac.id/welcome/detail/040411'+str(x),
            ]
            for url in urls:
                yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        yield{
            'judul':response.css('#content_journal > ul > li > div:nth-child(2) > a::text').extract(),
            'Penulis':response.css('#content_journal > ul > li > div:nth-child(2) > div:nth-child(2) > span::text').extract(),
            'Pembimbing 1':response.css('#content_journal > ul > li > div:nth-child(2) > div:nth-child(3) > span::text').extract(),
            'Pembimbing 2':response.css('#content_journal > ul > li > div:nth-child(2) > div:nth-child(4) > span::text').extract(),
            'Abstrak':response.css('#content_journal > ul > li > div:nth-child(4) > div:nth-child(2) > p::text').extract(),
            'Abstract':response.css('#content_journal > ul > li > div:nth-child(4) > div:nth-child(4) > p::text').extract()
        }
        

sebelum menjalankan code diatas harus menginstal terlebih dahulu framework scrapy dengan cara
buka cmd menggunakan run as administrator
setelah itu ketikkan

In [1]:
pip install scrapy

Note: you may need to restart the kernel to use updated packages.


code diatas digunakan untuk melakukan instalasi scrapy menggunakan cmd

### Melakukan Crawling Web

untuk menjalankannya pertama harus di save terlebih dahulu code diatas didalam file berformat py. setelah itu buka menggunakan visual studio code dengan cara open folder dimana file tersebut disimpan bukan open file tersebut. setelah folder tempat file crawling itu dibuka tinggal buka file nya setelah itu buka terminal dan ketikkan

**scrapy runspider nama_file.py -o data_awal_crawling.csv**

hasil dari save code diatas akan menghasilkan file berformat csv dimana akan ada banyak kolom kosong yang ada
oleh karena itu maka diperlukan sedikit modifikasi untuk menghapus kolom yang kosong dan sekaligus mengubah file dari csv ke xlsx
berikut ini kodenya

## Menghilangkan Kolom Kosong

In [49]:
import pandas as pd
data = pd.read_csv('hasil_crawling_awal.csv')

code diatas digunakan untuk mengimportkan library pandas yang akan digunakan untuk code selanjutnya. Lalu baris kedua digunakan untuk membaca file yang sudah di crawling.

In [50]:
data.dropna(inplace=True)
data.isnull().sum()

judul           0
Penulis         0
Pembimbing 1    0
Pembimbing 2    0
Abstrak         0
Abstract        0
dtype: int64

pada code di atas baris pertama digunakan untuk menghapus baris kosong, lalu baris kedua digunakan untuk medeteksi missing value kemudian dijumlahkan. setelah itu hasilnya di save dalam format xlsx dengan menggunakan code dibawah ini.

In [51]:
data.to_excel("hasil_crawling.xlsx")

## Latent Semantic Analysis (LSA)

Latent Semantic Analysis adalah suatu algoritma yang dapat digunakan untuk melakukan analisis pada frase/kalimat dalam sekumpulan dokumen, analisis yang dilakukan adalah menganalisa apakah setiap frase/kalimat memiliki hubungan antara lain atau tidak.Pada proses LSA akan menggunakan Singularity Value Decomposition (SVD atau dekomposisi nilai tunggal untuk mengambil informasi dan melakukan pengklasifikasian. kali ini yang dicari adalah hubungan setiap topik yang ada pada data hasil ceawling web yang sudah di lakukan preprocessing. berikut ini adalah langkah langkah nya

## Text Processing

Langkah ini sangat dibutuhkan dikarenakan data awal tersebut masih tidak teratur jadi akan diolah agar siap untuk diproses pada langkah selanjutnya. berikut ini adalah langkah langkah dari preprocessing

## Import Modules

In [159]:
# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
from openpyxl import load_workbook #library untuk menampilkan dokumen
#configure
#import nltk
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize

#preprocessing
from nltk.corpus import stopwords  #stopwords
from nltk import word_tokenize,sent_tokenize # tokenizing
from nltk.stem import PorterStemmer,LancasterStemmer  # using the Porter Stemmer and Lancaster Stemmer and others
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer  # lammatizer from WordNet

# for named entity recognition (NER)
from nltk import ne_chunk

# vectorizers for creating the document-term-matrix (DTM)
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

#stop-words
stop_words=set(nltk.corpus.stopwords.words('english'))

Code diatas digunakan untuk mengimport berbagai library yang akan digunakan pada code selanjutnya. 

## Loading Dataset

In [103]:
wr = load_workbook(filename = 'hasil_crawling.xlsx')
sheet_range = wr['Sheet1']

data = pd.DataFrame(sheet_range.values)
data.columns = ['','judul','Penulis', 'Pembimbing 1','Pembimbing 2', 'Abstrak', 'Abstract']


code diatas digunakan untuk membaca file yang bernama "hasil_crawling.xlsx". setelah itu pada baris ke 2 digunakan untuk mengambil hanya pada sheet 1 yang ada pada file excel. lalu langkah selanjutnya adalah untuk mengambil kolom pada baris pertama dengan nama yang sudah tertera pada baris ke 5.

In [104]:
data.head()

,,judul,Penulis,Pembimbing 1,Pembimbing 2,Abstrak,Abstract
0,NaN,judul,Penulis,Pembimbing 1,Pembimbing 2,Abstrak,Abstract
1,0.0,PENGARUH PERILAKU KONSUMEN TERHADAP KEPUTUSAN ...,Penulis : Hardik Pratama Suryanto,"Dosen Pembimbing I : Dr. H. Pribanus Wantara, ...","Dosen Pembimbing II :Dr. H. Muh. Syarif, Drs.,...",\nHardik Pratama S. 2014. Pengaruh Perilaku Ko...,ABSTRACT_x000D_\nHardik Pratama S. 2014. The I...
2,9.0,Bauran Pemasaran Terhadap Keputusan Pembelian ...,Penulis : Ervan Agus Feryanto,Dosen Pembimbing I : Dr.Ir.Nurita Andriani.MM,Dosen Pembimbing II :Nirma Kurriwati SP.M.Si,"Perkembangan dan kemajuan industri, salah satu...",The development and progress of industries can...
3,10.0,Pengaruh Komunikasi yang Efektif Dalam Organis...,Penulis : Zulsiamsyah Yuri,"Dosen Pembimbing I : Dr. Hj. Anugrahini SE., M...","Dosen Pembimbing II :Faidal SE.,MM.",Produktifitas karyawan sangat dibutuhkan perus...,Laborer productivity is really needed by compa...
4,19.0,PENGARUH KEMAMPUAN DAN MOTIVASI KARYAWAN TERHA...,Penulis : Lussiana Kurniati,Dosen Pembimbing I : Alkirom Wildan,Dosen Pembimbing II :Mudji Kuswinarno,Tujuan dari penelitian ini adalah mengetahui p...,The purpose of study is to determine the influ...


code diatas digunakan untuk menampilkan hasil dari file yang sudah dibaca pada code sebelumnya.

In [139]:
df = data[['Abstrak']]
df.head()

,Abstrak
0,Abstrak
1,\nHardik Pratama S. 2014. Pengaruh Perilaku Ko...
2,"Perkembangan dan kemajuan industri, salah satu..."
3,Produktifitas karyawan sangat dibutuhkan perus...
4,Tujuan dari penelitian ini adalah mengetahui p...


setelah itu membuat variabel yang hanya berisi kolom dari Abstrak.

## Data Cleaning dan Preprocessing Teks

pada langkah ini digunakan untuk menghilangkan kata tambahan yaitu kata yang karakternya kurang dari 3 angka.

In [140]:
def clean_text(headline):
  le=WordNetLemmatizer()
  word_tokens=word_tokenize(headline)
  tokens=[le.lemmatize(w) for w in word_tokens if w not in stop_words and len(w)>3]
  cleaned_text=" ".join(tokens)
  return cleaned_text
  
  

pada code disini adalah membuat sebuah fungsi yang berfungsi untuk melakukan data cleaning.

In [156]:
df_bersih = df['Abstrak'].apply(clean_text)
df_bersih.head()

0                                              Abstrak
1    Hardik Pratama 2014 Pengaruh Perilaku Konsumen...
2    Perkembangan kemajuan industri salah satunya k...
3    Produktifitas karyawan sangat dibutuhkan perus...
4    Tujuan dari penelitian adalah mengetahui penga...
Name: Abstrak, dtype: object

disini dapat terlihat perbedaan dari kata sebelum dilakukan preprocessing dan sesudah dimana kata ada dihilangkan.

In [137]:
df = df_bersih
df.head()

0                                              Abstrak
1    Hardik Pratama 2014 Pengaruh Perilaku Konsumen...
2    Perkembangan kemajuan industri salah satunya k...
3    Produktifitas karyawan sangat dibutuhkan perus...
4    Tujuan dari penelitian adalah mengetahui penga...
Name: Abstrak, dtype: object

# Menghilangkan Angka

In [68]:
import re
dokumenre=[]
for i in df:
    hasil = re.sub(r"\d+", "", i)
    dokumenre.append(hasil) 

In [69]:
dokumen2=[]
for i in dokumenre:
    hasil =  i.replace('\n','') 
    dokumen2.append(hasil) 
print(dokumen2)

['Abstrak', 'Hardik Pratama  Pengaruh Perilaku Konsumen Terhadap Keputusan Pembelian Makanan Tradisional Kabupaten Bangkalan bawah bimbingan Pribanus Wantara Drs. Syarif Drs.Ec. Si._xD_ Tujuan penelitian adalah untuk mengetahui pengaruh perilaku konsumen yang terdiri dari kebudayaan sosial pribadi psikologis secara simultan terhadap keputusan pembelian makanan tradisional kabupaten Bangkalan untuk mengetahui variabel perilaku konsumen yang berpengaruh dominan terhadap keputusan pembelian makanan tradisional kabupaten Bangkalan._xD_ Penelitian menggunakan pendekatan pendekatan kuantitatif yang berusaha untuk mengetahui bagaimana pengaruh perilaku konsumen terhadap keputsan pembelian makanan tradisional kabupaten Bangkalan Teknik pengambilan sampel menggunakan purposive sampling dengan responden Teknik pengumpulan data yaitu dengan wawancara kuesioner menggunakan skala likert untuk mengukur jawaban responden Untuk menguji instrumen menggunakan validitas reliabilitas asumsi klasik Sedangk

beberapa kode diatas digunakan dalam keadaan berurutan, yang berfungsi untuk menghilangkan angka pada setiap kata yang ada. langkah ini menggunakan library re untuk melaksanakannya

# Menghilangkan kata hubung


In [66]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from nltk.tokenize import word_tokenize
factory = StopWordRemoverFactory()
stopword  = factory.create_stop_word_remover()

In [70]:
a=len(dokumen2)
dokumenstop=[]
for i in range(0, a):
    sentence = stopword.remove(dokumen2[i])
    dokumenstop.append(sentence)
print(dokumenstop)   

['Abstrak', 'Hardik Pratama  Pengaruh Perilaku Konsumen Terhadap Keputusan Pembelian Makanan Tradisional Kabupaten Bangkalan bawah bimbingan Pribanus Wantara Drs. Syarif Drs.Ec. Si._xD_ Tujuan penelitian mengetahui pengaruh perilaku konsumen terdiri kebudayaan sosial pribadi psikologis simultan keputusan pembelian makanan tradisional kabupaten Bangkalan mengetahui variabel perilaku konsumen berpengaruh dominan keputusan pembelian makanan tradisional kabupaten Bangkalan._xD_ Penelitian menggunakan pendekatan pendekatan kuantitatif berusaha mengetahui bagaimana pengaruh perilaku konsumen keputsan pembelian makanan tradisional kabupaten Bangkalan Teknik pengambilan sampel menggunakan purposive sampling responden Teknik pengumpulan data wawancara kuesioner menggunakan skala likert untuk mengukur jawaban responden Untuk menguji instrumen menggunakan validitas reliabilitas asumsi klasik Sedangkan teknik analisis data menggunakan analisis regresi linier berganda variabel digunakan lebih varia

lalu pada kode diatas digunakan untuk menghilangkan kata hubung. kode diatas menggunakan library sastrawi agar dapat berkerja, sastrawi juga digunakan karena library ini support untuk kata kata bahasa indonesia.

# Penggunaan Sastrawi untuk menghilangkan tanda baca

In [71]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from nltk.tokenize import word_tokenize
import string
factory = StopWordRemoverFactory()
dokumenst=[]
for i in dokumenstop:
    output = i.translate(str.maketrans("","",string.punctuation))
    dokumenst.append(output)
    
print(dokumenst)

['Abstrak', 'Hardik Pratama  Pengaruh Perilaku Konsumen Terhadap Keputusan Pembelian Makanan Tradisional Kabupaten Bangkalan bawah bimbingan Pribanus Wantara Drs Syarif DrsEc SixD Tujuan penelitian mengetahui pengaruh perilaku konsumen terdiri kebudayaan sosial pribadi psikologis simultan keputusan pembelian makanan tradisional kabupaten Bangkalan mengetahui variabel perilaku konsumen berpengaruh dominan keputusan pembelian makanan tradisional kabupaten BangkalanxD Penelitian menggunakan pendekatan pendekatan kuantitatif berusaha mengetahui bagaimana pengaruh perilaku konsumen keputsan pembelian makanan tradisional kabupaten Bangkalan Teknik pengambilan sampel menggunakan purposive sampling responden Teknik pengumpulan data wawancara kuesioner menggunakan skala likert untuk mengukur jawaban responden Untuk menguji instrumen menggunakan validitas reliabilitas asumsi klasik Sedangkan teknik analisis data menggunakan analisis regresi linier berganda variabel digunakan lebih variabel denga

sama dengan kode sebelumnya kode ini digunakan untuk 

setelah itu jadikan df_bersih sebagai df lalu ditampilkan

## Melakukan Ekstraksi Fitur dan Membuat Dokumen term Matrix

dokumen term matrix atau disingkan DTM adalah representasi dari dokumen dalam corpus. DTM akan mempresentasikan dokumen dalam struktur numerik. melalui DTM kita dapat melakukan analisis yang lebih menarik.

In [78]:
vect =TfidfVectorizer(stop_words=stop_words,max_features=1000) # to play with. min_df,max_df,max_features etc...
vect_text=vect.fit_transform(dokumenst)
print(vect_text.shape)
print(vect_text)

(30, 905)
  (0, 1)	1.0
  (1, 900)	0.02083318912701066
  (1, 573)	0.05399580353657585
  (1, 382)	0.0662762533413926
  (1, 304)	0.0237928769955213
  (1, 496)	0.027348148476146614
  (1, 238)	0.0237928769955213
  (1, 871)	0.05004238172655131
  (1, 125)	0.03962859828096337
  (1, 407)	0.041715353731759094
  (1, 142)	0.0237928769955213
  (1, 73)	0.0246170695089527
  (1, 418)	0.02836082089518089
  (1, 685)	0.0246170695089527
  (1, 23)	0.04166637825402132
  (1, 727)	0.03453175110395285
  (1, 361)	0.05004238172655131
  (1, 37)	0.05004238172655131
  (1, 687)	0.046812200908769606
  (1, 882)	0.046812200908769606
  (1, 271)	0.05399580353657585
  (1, 484)	0.046812200908769606
  (1, 286)	0.05004238172655131
  (1, 485)	0.046812200908769606
  (1, 876)	0.05096296423383561
  :	:
  (29, 496)	0.022560869814094962
  (29, 238)	0.058883840074247504
  (29, 142)	0.019627946691415835
  (29, 73)	0.02030786474925057
  (29, 418)	0.023396274471565073
  (29, 685)	0.02030786474925057
  (29, 23)	0.03437270118061988
  (2

dengan menggunakan code diatas kita dapat melihat kata kata yang paling sering keluar dan juga kata kata yang jarang keluar dengan cara memperhatikan seberapa besar skor idf. semakin kecil nilai dari skor idf berarti semakin sering muncul kata tersebut.

# K Means Clustering

In [93]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from scipy.sparse.csr import csr_matrix

vectorizer = TfidfVectorizer(input=dokumenst)

X = vectorizer.fit_transform(dokumenst)
print(X.todense())

[[0.         1.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.0448034 ]
 [0.         0.         0.03115173 ... 0.         0.         0.04874851]]


In [79]:
''''idf=vect.idf_
dd=dict(zip(vect.get_feature_names_out(), idf))
l=sorted(dd, key=(dd).get)
# print(l)
print(l[0],l[-1])
print(dd['kemajuan'])
print(dd['sangat'])

penelitian yaitu
3.740840023925201
2.8245492920510458


code di atas digunakan untuk melakukan extraksi fitur dari vectorizer. lalu pada 2 baris terbawah digunakan untuk menampilkan kata dari gerak dan hidup.

In [80]:
''''from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [81]:
''''def preprocessing(line):
    line = line.lower()
    line = re.sub(r"[{}]".format(string.punctuation), " ", line)
    return line

In [82]:
'''''#tfidf_vectorizer = TfidfVectorizer(preprocessor=preprocessing)
#tfidf = tfidf_vectorizer.fit_transform(all_text)

#vect =TfidfVectorizer(stop_words=stop_words,max_features=1000) # to play with. min_df,max_df,max_features etc...
#vect_text=vect.fit_transform(df)

kmeans = KMeans(n_clusters=2).fit(vect_text)

In [91]:
''''
lines_for_predicting = ["kemajuan teknologi", "teknik analisis"]
k = kmeans.predict(vect.transform(lines_for_predicting))

# array([0, 1], dtype=int32)

## Topic Modelling Menggunakan LSA

Pada langkah sebelumnya adalah langkah langkah agar teks siap diproses pada langkah ini. Jadi pada langkah ini adalah langkah untuk melakukan pendekatan LSA. Pada dasarnya LSA adalah dekomposisi nilai tunggal

In [95]:
from sklearn.decomposition import TruncatedSVD
lsa_model = TruncatedSVD(n_components=10, algorithm='randomized', n_iter=10, random_state=42)

lsa_top=lsa_model.fit_transform(X)


pada code diatas digunakan untuk memproses dekomposisi kalimat yang ada di Abstrak menggunakan TruncatedSVD lalu disimpan di dalam variabel bernama lsa_top

In [96]:
print(lsa_top)
print(lsa_top.shape)  # (no_of_doc*no_of_topics)

[[ 5.84783688e-03 -1.40019532e-02 -3.65464815e-02 -3.30116063e-02
  -2.59207380e-02 -5.15933799e-02  1.01094170e-01  5.50073498e-02
   4.27990103e-01  4.36419145e-01]
 [ 4.52212289e-01  2.50371036e-01  1.23908128e-01 -1.80061576e-01
   3.56151704e-01 -2.40669112e-01 -2.06746073e-01  1.84319483e-02
   1.76353232e-01 -4.80813144e-02]
 [ 5.38849668e-01  3.82341733e-01 -1.49519498e-01  1.85092593e-01
  -2.67648520e-01  5.89294195e-02  6.54253953e-03  5.97221843e-03
  -1.20988284e-01  9.83422355e-02]
 [ 2.13720623e-01 -3.01620361e-01 -2.68181324e-01  2.87840416e-01
   2.14476286e-01 -1.03730897e-01 -1.10699295e-01  9.26975226e-02
  -1.58230224e-01 -9.00428806e-02]
 [ 3.25216438e-01 -3.86407472e-01 -3.37586633e-01 -1.96222999e-01
  -5.96582665e-03 -2.42912120e-01  1.08584662e-01 -9.88998096e-02
  -1.37143284e-01 -8.52530161e-02]
 [ 2.87589640e-01  1.26842258e-01  4.63086573e-02 -7.55301022e-03
   1.07866209e-01  2.77934491e-02  5.30872358e-02 -9.12446148e-03
   2.82317446e-01 -2.60288682e-01

setelah itu dipanggil untuk ditampilkan seperti pada code diatas.

In [97]:
l=lsa_top[0]
print("Document 0 :")
for i,topic in enumerate(l):
  print("Topic ",i," : ",topic*100)

Document 0 :
Topic  0  :  0.5847836879939213
Topic  1  :  -1.4001953216046097
Topic  2  :  -3.6546481454772217
Topic  3  :  -3.3011606261433983
Topic  4  :  -2.5920737984100652
Topic  5  :  -5.15933798734842
Topic  6  :  10.109417040899727
Topic  7  :  5.500734983370039
Topic  8  :  42.79901027761908
Topic  9  :  43.64191448928669


pada code diatas yang dipanggil hanya pada abstrak ke 1 atau pada code menggunakan index ke 0

In [98]:
print(lsa_model.components_.shape) # (no_of_topics*no_of_words)
print(lsa_model.components_)

(10, 905)
[[ 0.00408816  0.00584784  0.03218957 ...  0.00590165  0.00349141
   0.00855317]
 [-0.00533497 -0.01400195 -0.0147126  ... -0.0102549  -0.00348976
  -0.01193013]
 [ 0.01199273 -0.03654648  0.03202769 ... -0.00560592  0.00549563
  -0.00347077]
 ...
 [ 0.00155921  0.05500735  0.02730259 ... -0.00903148  0.00678123
  -0.00737162]
 [ 0.00457763  0.4279901  -0.01513773 ... -0.00353066  0.01033061
   0.00174788]
 [ 0.01008189  0.43641914  0.02363544 ... -0.00139902 -0.00314761
  -0.0064562 ]]


setelah itu ditampilkan seperti pada code diatas

In [99]:
# most important words for each topic
vocab = vect.get_feature_names_out()

for i, comp in enumerate(lsa_model.components_):
    vocab_comp = zip(vocab, comp)
    sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:10]
    print("Topic "+str(i)+": ")
    for t in sorted_words:
        print(t[0],end=" ")
    print("\n")

Topic 0: 
variabel keputusan pembelian penelitian kerja produk kinerja konsumen promosi pengaruh 

Topic 1: 
keputusan pembelian produk promosi konsumen merek pemasaran bauran distribusi harga 

Topic 2: 
kepemimpinan nilai variabel kabupaten pegawai perilaku pengujian tipe kompensasi hipotesis 

Topic 3: 
kerja produktivitas prestasi distribusi komunikasi kesehatan keselamatan kecamatan produk area 

Topic 4: 
merek kualitas kepercayaan produktivitas konsumen psikologis kemudahan kerja perilaku komunikasi 

Topic 5: 
perputaran saham ratio profitabilitas perusahaan split stock kualitas kepuasan terdaftar 

Topic 6: 
kualitas kepercayaan kepuasan komitmen hubungan merk dinamika multi patner kompetensi 

Topic 7: 
saham kepemilikan pegawai peningkatan split stock organisasi institusional kepuasan perbedaan 

Topic 8: 
abstrak kompetensi pesan perspektif kepuasan dosen kabupaten kelurahan pegawai kerja 

Topic 9: 
abstrak kepercayaan kepemimpinan kualitas prestasi kemudahan komitmen dina

ketika code diatas itu dijalankan maka akan tampil topic topic seperti pada contoh diatas. lalu kita dapat menganalisis hasil dari code tersebut untuk langkah selanjutnya.